In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
import csv

In [2]:
import pandas as pd
from datetime import datetime, date

In [3]:
### Noms des actions capitalisées au CAC 40
page = requests.get("https://finance.yahoo.com/quote/%5EFCHI/components")
soup = BeautifulSoup(page.content, 'html.parser')
tableau = list(soup.find_all('tbody'))
tableau = tableau[0]
tab_content = list(tableau.children)
n = len(tab_content)
lines_list = list()
for i in range(n):
    lines_list.append(list(tab_content[i].children))


In [4]:
Stocks = ["AC.PA", "AI.PA", "AIR.PA", "ATO.PA", "CS.PA", "BNP.PA", "EN.PA", "CAP.PA", "CA.PA", "ACA.PA", 
          "BN.PA", "ENGI.PA", "EI.PA", "KER.PA", "OR.PA", "LHN.PA", "LR.PA", "MC.PA", "ML.PA", "ORA.PA",
          "RI.PA", "UG.PA", "PUB.PA", "RNO.PA", "SAF.PA", "MT.AS", "UL.AS", "SOLB.BR", "SGO.PA", "SAN.PA",
          "SU.PA", "GLE.PA", "SW.PA", "STM.PA", "FTI.PA", "FP.PA", "FR.PA", "VIE.PA", "DG.PA", "VIV.PA"]

In [5]:
Stocks = list()
for i in range(len(lines_list)):
    temp = []
    for j in range(len(lines_list[i])):
        temp.append(lines_list[i][j].get_text())
    Stocks.append(temp[0])

In [6]:
header = ["Date", "Open", "High", "Low", "Close", "Adj close", "Volume"]

In [7]:
browser = webdriver.Chrome('D:/Centrale/Cours/2A/Projet_Inno_S8/Github/projets8/chromedriver.exe')

In [8]:
for stock in Stocks:
    ### Les liens à scraper
    link = "https://finance.yahoo.com/quote/" + stock + "/history?period1=1364338800&period2=1522101600&interval=1d&filter=history&frequency=1d"
    
    
    browser.get(link)
    #Nous allons scroller la page jusqu'à l'arret
    lenOfPage = browser.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);var lenOfPage=document.documentElement.scrollHeight;return lenOfPage;")
    match=False    #match indique si on est arrivé au bout de la page ou non
    while(match==False):
        lastCount = lenOfPage
        time.sleep(0.1)
        #Code en entrée en JavaScript
        lenOfPage = browser.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);var lenOfPage=document.documentElement.scrollHeight;return lenOfPage;")
        if lastCount==lenOfPage:
            match=True

    #Maintenant nous pouvons récupérer le code de la page avec toutes les données voulues
    source_data = browser.page_source

    #On peut à present utiliser BeautifulSoup pour parser la page
    soup = BeautifulSoup(source_data, 'html.parser')

    #On récupère le tableau
    tableau = list(soup.find_all('tbody'))
    tableau = tableau[1]
    tab_content = list(tableau.children)

    #On récupère les lignes du tableau 
    n = len(tab_content)
    lines_list = list()
    for i in range(n):
        lines_list.append(list(tab_content[i].children))

    #On crée le tableau des données historiques
    historical = list()
    for i in range(len(lines_list)):
        temp = []
        for j in range(len(lines_list[i])):
            temp.append(lines_list[i][j].get_text())
        historical.append(temp)

    
    """Traitement du contenu du tableau"""
    n = len(historical)
    i = 0
    compteur = 0
    while compteur < n:
        temp = historical[i]
        if len(temp)>2:
            temp[0] = temp[0].replace(',','')
            temp[0] = datetime.strptime(temp[0] , "%b %d %Y").date()
            c = temp[-1]
            if c == '-':
                historical.remove(temp)
                compteur +=1
                continue
            else:
                c = int(c.replace(',',''))
                temp[-1] = c
            for j in range(1,6):
                if temp[j] == '-':
                    historical.remove(temp)
                    i-=1
                    break
                else:
                    temp[j] = float(temp[j])
            i+=1
        else:
            historical.remove(temp)
        compteur +=1
    
    ### Transformation en DataFrame
    Frame = pd.DataFrame(data = historical, columns = header)
    
    ### Sortie d'un CSV
    Frame.to_csv(path_or_buf = stock + ".csv", header = True, index = False)

WebDriverException: Message: chrome not reachable
  (Session info: chrome=66.0.3359.139)
  (Driver info: chromedriver=2.35.528161 (5b82f2d2aae0ca24b877009200ced9065a772e73),platform=Windows NT 10.0.16299 x86_64)


In [8]:
browser.get("https://finance.yahoo.com/quote/BNP.PA/history?period1=1362351600&period2=1520118000&interval=1d&filter=history&frequency=1d")
#Nous allons scroller la page jusqu'à l'arret
lenOfPage = browser.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);var lenOfPage=document.documentElement.scrollHeight;return lenOfPage;")
match=False    #match indique si on est arrivé au bout de la page ou non
while(match==False):
    lastCount = lenOfPage
    time.sleep(0.1)
    #Code en entrée en JavaScript
    lenOfPage = browser.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);var lenOfPage=document.documentElement.scrollHeight;return lenOfPage;")
    if lastCount==lenOfPage:
        match=True

#Maintenant nous pouvons récupérer le code de la page avec toutes les données voulues
source_data = browser.page_source

#On peut à present utiliser BeautifulSoup pour parser la page
soup = BeautifulSoup(source_data, 'html.parser')

#On récupère le tableau
tableau = list(soup.find_all('tbody'))
tableau = tableau[1]
tab_content = list(tableau.children)

#On récupère les lignes du tableau 
n = len(tab_content)
lines_list = list()
for i in range(n):
    lines_list.append(list(tab_content[i].children))

#On crée le tableau des données historiques
historical = list()
for i in range(len(lines_list)):
    temp = []
    for j in range(len(lines_list[i])):
        
        temp.append(lines_list[i][j].get_text())
    historical.append(temp)

    
"""Traitement du contenu du tableau"""
n = len(historical)
i = 0
compteur = 0
while compteur < n:
    temp = historical[i]
    if len(temp)>2:
        temp[0] = temp[0].replace(',','')
        temp[0] = datetime.strptime(temp[0] , "%b %d %Y").date()
        for j in range(1,6):
            temp[j] = float(temp[j])
        c = temp[-1]
        if c == '-':
            temp[-1] = None
        else:
            c = int(c.replace(',',''))
            temp[-1] = c
        i+=1
    else:
        historical.remove(temp)
    compteur +=1

In [1]:
help(continue)

SyntaxError: invalid syntax (<ipython-input-1-0d7ba4101899>, line 1)